# Ultimate Inc. Take Home Challenge
# Michael Phillips

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [43]:
df = pd.read_json('logins.json')
df.head()

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93142 entries, 0 to 93141
Data columns (total 1 columns):
login_time    93142 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 727.8 KB


In [49]:
df['Date'] = df['login_time'].dt.date
df['Time'] = df['login_time'].dt.time

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93142 entries, 0 to 93141
Data columns (total 3 columns):
login_time    93142 non-null datetime64[ns]
Date          93142 non-null object
Time          93142 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 2.1+ MB


In [60]:
df['login_time'] = pd.to_datetime(df['login_time'], format='%Y/%m/%d %H:%M:%S')
df.set_index(df['login_time'], inplace=True)
df['count'] = 1
df.head()

,login_time,Date,Time,count
login_time,,,,
1970-01-01 20:13:18,1970-01-01 20:13:18,1970-01-01,20:13:18,1
1970-01-01 20:16:10,1970-01-01 20:16:10,1970-01-01,20:16:10,1
1970-01-01 20:16:37,1970-01-01 20:16:37,1970-01-01,20:16:37,1
1970-01-01 20:16:36,1970-01-01 20:16:36,1970-01-01,20:16:36,1
1970-01-01 20:26:21,1970-01-01 20:26:21,1970-01-01,20:26:21,1


In [59]:
df1 = df.groupby(pd.TimeGrouper(freq='15min'))

In [62]:
df.resample('15T').sum()

,count
login_time,
1970-01-01 20:00:00,2.0
1970-01-01 20:15:00,6.0
1970-01-01 20:30:00,9.0
1970-01-01 20:45:00,7.0
1970-01-01 21:00:00,1.0
1970-01-01 21:15:00,4.0
1970-01-01 21:30:00,NaN
1970-01-01 21:45:00,4.0
1970-01-01 22:00:00,9.0
